Read Data

In [1]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_mldata
from sklearn.preprocessing import StandardScaler
from sklearn.utils import check_random_state 
import numpy as np

mnist = fetch_mldata('MNIST original')
X = mnist.data.astype('float64')
y = mnist.target
random_state = check_random_state(0) 

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=1000, test_size=300, random_state= random_state)

X_train = X_train.reshape(X_train.shape[0], 1, 28, 28)

In [2]:
def Pooling(tensor):
    new = np.zeros((tensor.shape[0], int(tensor.shape[1]/2), int(tensor.shape[2]/2) ))
    for i in range(new.shape[0]):
        for j in range(new.shape[1]):
            for k in range(new.shape[2]):
                new[i,j,k] = np.mean(tensor[i, 2*j:(2*j+2), 2*k:(2*k+2)])
    return new
def Conv(tensor, filter, bias):
    slide = tensor.shape[1] +1-5
    new = np.zeros((filter.shape[3],slide,slide))
    for kk in range(filter.shape[3]):
        for i in range(slide):
            for j in range(slide):
                sub = 0
                for m in range(filter.shape[0]):
                    tempa = tensor[m, i:i+5, i:i+5].reshape(5,5)
                    tempb = filter[m, :, :, kk].reshape(5,5)                    
                    temp = tempa * tempb
                    sub += np.sum(temp)
                new[kk,i,j] = sub       
    new += bias
    return new

In [4]:
#LeNet
#Inialize
Filter0 =  np.random.random( (1, 5, 5, 6))
b0 = np.random.random((6,28,28))
Filter2 =  np.random.random( (6, 5, 5, 16) )
b2 = np.random.random((16,10,10))
Filter4 =  np.random.random( (16, 5, 5, 120) )
b4 = np.random.random((120,1,1))
W5 = np.random.random( (84,120 ) )
b5 = np.random.random( (84,1)  )

W6 = np.random.random((10,84))
b6 = np.random.random( (10,1) )

Input = np.pad(X_train, ((0,0),(0,0), (2,2),(2,2)) , 'constant' )

# for i, image in enumerate(Input): 
image = Input[0]
C1 = Conv(image, Filter0, b0)
S2 = Pooling(C1)
C3 = Conv(S2, Filter2, b2)
S4 = Pooling(C3)
C5 = Conv(S4, Filter4, b4)
C5 = C5.reshape(120,1)
F6 = np.maximum(np.dot(W5, C5)+b5, 0)
F7 = np.dot(W6,F6) + b6